# Get SPACE TRACK data

In [142]:
%load_ext lab_black

The lab_black extension is already loaded. To reload it, use:
  %reload_ext lab_black


In [149]:
import pandas as pd
import geopandas as gpd
import altair as alt
from datetime import timedelta
import numpy as np
import glob

In [150]:
pd.options.display.max_columns = 1000
pd.options.display.max_rows = 1000
pd.options.display.max_colwidth = None

### API config

In [151]:
# username = 'mstiles@grid.news'
# password = 'thespacerace123'
# output = 'json'

In [156]:
!curl -c 'params/cookies.txt' -b 'params/cookies.txt' 'https://www.space-track.org/ajaxauth/login' -d 'identity=mstiles@grid.news&password=thespacerace123'

""

### Boxscore

In [157]:
# !curl --limit-rate 100K --cookie 'params/cookies.txt' 'https://www.space-track.org/basicspacedata/query/class/boxscore' -o 'data/raw/boxscore.json'

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 35074  100 35074    0     0  33204      0  0:00:01  0:00:01 --:--:-- 33372


In [158]:
boxscore = pd.read_json("data/raw/boxscore.json")

In [159]:
boxscore.columns = boxscore.columns.str.lower()

In [160]:
boxscore = boxscore[boxscore["country"] != "ALL"]

In [161]:
boxscore.head()

,country,spadoc_cd,orbital_tba,orbital_payload_count,orbital_rocket_body_count,orbital_debris_count,orbital_total_count,decayed_payload_count,decayed_rocket_body_count,decayed_debris_count,decayed_total_count,country_total
0,ARAB SATELLITE COMMUNICATIONS ORGANIZATION,AB,0,14,0,0,14,1,0,0,1,15
1,ASIASAT CORP,AC,0,8,0,0,8,0,0,0,0,8
2,REPUBLIC OF ANGOLA,AGO,0,1,0,0,1,0,0,0,0,1
3,ALGERIA,ALG,0,6,0,0,6,0,0,0,0,6
4,ARGENTINA,ARGN,0,39,0,1,40,2,0,0,2,42


In [181]:
boxscore_slim = boxscore[
    [
        "country",
        "spadoc_cd",
        "orbital_payload_count",
        "orbital_rocket_body_count",
        "orbital_debris_count",
        "orbital_total_count",
        "decayed_total_count",
        "country_total",
    ]
].sort_values("orbital_debris_count", ascending=False)
boxscore_slim.head(4)

,country,spadoc_cd,orbital_payload_count,orbital_rocket_body_count,orbital_debris_count,orbital_total_count,decayed_total_count,country_total
18,COMMONWEALTH OF INDEPENDENT STATES,CIS,1551,1048,5984,8588,15609,24197
106,UNITED STATES OF AMERICA,US,4001,726,4475,9219,6833,16052
77,PEOPLES REPUBLIC OF CHINA,PRC,517,195,3659,4453,1530,5983
32,FRANCE,FR,80,165,355,600,756,1356


In [163]:
boxscore["orbital_payload_count"].sum()

8170

In [164]:
countries = boxscore[["country", "spadoc_cd"]]

In [165]:
countries.head()

,country,spadoc_cd
0,ARAB SATELLITE COMMUNICATIONS ORGANIZATION,AB
1,ASIASAT CORP,AC
2,REPUBLIC OF ANGOLA,AGO
3,ALGERIA,ALG
4,ARGENTINA,ARGN


---

### Geosynchronous Report
This shows objects with 1430 <= period <= 1450 minutes.

In [166]:
# !curl --limit-rate 100K --cookie 'params/cookies.txt'  'https://www.space-track.org/basicspacedata/query/class/satcat/PERIOD/1430--1450/CURRENT/Y/DECAY/null-val/orderby/NORAD_CAT_ID/format/json'  -o 'data/raw/geo.json'

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  466k  100  466k    0     0   100k      0  0:00:04  0:00:04 --:--:--  109k    0 --:--:-- --:--:-- --:--:--     0


In [167]:
geo_df = pd.read_json("data/raw/geo.json")

In [168]:
geo_df.head()

,INTLDES,NORAD_CAT_ID,OBJECT_TYPE,SATNAME,COUNTRY,LAUNCH,SITE,DECAY,PERIOD,INCLINATION,APOGEE,PERIGEE,COMMENT,COMMENTCODE,RCSVALUE,RCS_SIZE,FILE,LAUNCH_YEAR,LAUNCH_NUM,LAUNCH_PIECE,CURRENT,OBJECT_NAME,OBJECT_ID,OBJECT_NUMBER
0,1963-031A,634,PAYLOAD,SYNCOM 2 (A 26),US,1963-07-26,AFETR,NaN,1436.17,33.21,35815,35761,NaN,NaN,0,LARGE,7956,1963,31,A,Y,SYNCOM 2 (A 26),1963-031A,634
1,1964-047A,858,PAYLOAD,SYNCOM 3,US,1964-08-19,AFETR,NaN,1436.91,3.07,35805,35800,NaN,NaN,0,LARGE,7957,1964,47,A,Y,SYNCOM 3,1964-047A,858
2,1965-028A,1317,PAYLOAD,INTELSAT 1-F1,ITSO,1965-04-06,AFETR,NaN,1435.42,3.27,35787,35759,NaN,NaN,0,MEDIUM,7957,1965,28,A,Y,INTELSAT 1-F1,1965-028A,1317
3,1966-110A,2608,PAYLOAD,ATS 1,US,1966-12-07,AFETR,NaN,1435.52,1.03,35789,35762,NaN,NaN,0,LARGE,7956,1966,110,A,Y,ATS 1,1966-110A,2608
4,1967-001A,2639,PAYLOAD,INTELSAT 2-F2,ITSO,1967-01-11,AFETR,NaN,1435.21,0.85,35798,35740,NaN,NaN,0,LARGE,7957,1967,1,A,Y,INTELSAT 2-F2,1967-001A,2639


In [169]:
len(geo_df)

1026

---

### Everything? 

In [170]:
# !curl --limit-rate 100K --cookie cookies.txt  'https://www.space-track.org/basicspacedata/query/class/satcat/format/json'  -O 'all.json'

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 22.9M  100 22.9M    0     0    99k      0  0:03:55  0:03:55 --:--:--  100k    0  0:03:54  0:00:38  0:03:16   99k 0:00:56  0:02:58   99k   0  0:03:55  0:01:09  0:02:46   99k    0   100k      0  0:03:55  0:01:42  0:02:13   99k 100k      0  0:03:55  0:02:28  0:01:27   99k   0     0   100k      0  0:03:55  0:02:32  0:01:23  100k  0:03:55  0:03:31  0:00:24   99k
curl: (6) Could not resolve host: all.json


In [171]:
all_df = pd.read_json("data/raw/all.json")

In [172]:
all_df.columns = all_df.columns.str.lower()

In [173]:
len(all_df)

In [204]:
all_df

,intldes,norad_cat_id,object_type,satname,country,launch,site,decay,period,inclination,apogee,perigee,comment,commentcode,rcsvalue,rcs_size,file,launch_year,launch_num,launch_piece,current,object_name,object_id,object_number
0,1957-001A,1,ROCKET BODY,SL-1 R/B,CIS,1957-10-04,TTMTR,1957-12-01,96.19,65.10,938.0,214.0,None,4.0,0,LARGE,1,1957,1,A,Y,SL-1 R/B,1957-001A,1
1,1958-001A,4,PAYLOAD,EXPLORER 1,US,1958-02-01,AFETR,1970-03-31,88.48,33.15,215.0,183.0,None,NaN,0,None,1,1958,1,A,Y,EXPLORER 1,1958-001A,4
2,1958-003A,6,PAYLOAD,EXPLORER 3,US,1958-03-26,AFETR,1958-06-28,103.60,33.50,1739.0,117.0,None,5.0,0,None,1,1958,3,A,Y,EXPLORER 3,1958-003A,6
3,1958-004A,7,ROCKET BODY,SL-1 R/B,CIS,1958-05-15,TTMTR,1958-12-03,102.74,65.14,1571.0,206.0,None,NaN,0,None,1,1958,4,A,Y,SL-1 R/B,1958-004A,7
4,1958-004B,8,PAYLOAD,SPUTNIK 3,CIS,1958-05-15,TTMTR,1960-04-06,88.43,65.06,255.0,139.0,None,NaN,0,LARGE,1,1958,4,B,Y,SPUTNIK 3,1958-004B,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51420,1998-067TB,51439,TBA,OBJECT TB,TBD,1998-11-20,TTMTR,None,92.86,51.64,420.0,409.0,None,NaN,0,None,7957,1998,67,TB,Y,OBJECT TB,1998-067TB,51439
51421,1998-067TC,51440,TBA,OBJECT TC,TBD,1998-11-20,TTMTR,None,92.84,51.64,420.0,408.0,None,NaN,0,None,7957,1998,67,TC,Y,OBJECT TC,1998-067TC,51440
51422,1998-067TD,51441,TBA,OBJECT TD,TBD,1998-11-20,TTMTR,None,92.87,51.64,421.0,409.0,None,NaN,0,None,7957,1998,67,TD,Y,OBJECT TD,1998-067TD,51441
51423,1998-067TE,51442,TBA,OBJECT TE,TBD,1998-11-20,TTMTR,None,92.86,51.64,421.0,409.0,None,NaN,0,None,7957,1998,67,TE,Y,OBJECT TE,1998-067TE,51442


In [174]:
all_df.object_type.value_counts()

DEBRIS         32913
PAYLOAD        11939
ROCKET BODY     6304
TBA              269
Name: object_type, dtype: int64

In [182]:
all_df.rcs_size.value_counts()

SMALL     22230
LARGE     11712
MEDIUM     7409
Name: rcs_size, dtype: int64

In [206]:
inactive = all_df[~all_df["decay"].isnull()]

In [210]:
len(inactive)

25992

In [207]:
all_grouped = (
    inactive.groupby(["launch_year", "object_type"]).size().reset_index(name="count")
)

In [208]:
all_grouped.head()

,launch_year,object_type,count
0,1957,PAYLOAD,2
1,1957,ROCKET BODY,1
2,1958,PAYLOAD,7
3,1958,ROCKET BODY,1
4,1959,DEBRIS,4


In [209]:
alt.Chart(all_grouped).mark_bar().encode(
    x="launch_year:O",
    y="count",
    facet=alt.Facet("object_type", columns=2),
).properties(width=500, height=300)

alt.Chart(...)